In [3]:
# Params

IMGS_ORIGIN = "./original_imgs/"
DATASET_DIR = "./dataset/"
TRAIN_DIR = DATASET_DIR + "train/"
VALIDATION_DIR = DATASET_DIR + "validation/"
TEST_DIR = DATASET_DIR + "test/"
TRAIN_DOGS_PATH = DATASET_DIR + "train/dogs/"
TRAIN_CATS_PATH = DATASET_DIR + "train/cats/"
VALIDATION_DOGS_PATH = DATASET_DIR + "validation/dogs/"
VALIDATION_CATS_PATH = DATASET_DIR + "validation/cats/"
TEST_DOGS_PATH = DATASET_DIR + "test/dogs/"
TEST_CATS_PATH = DATASET_DIR + "test/cats/"

In [5]:
import os
import h5py
import matplotlib.pyplot as plt
import tensorflow
from keras import layers, models, optimizers
from keras.preprocessing.image import ImageDataGenerator
import glob
import shutil

ModuleNotFoundError: No module named 'tensorflow'

In [10]:
# Prepare Dataset


for i in range(0,12000):
    train_dogs = glob.glob(IMGS_ORIGIN + "train/dog." + str(i) + ".jpg")
    train_cats = glob.glob(IMGS_ORIGIN + "train/cat." + str(i) + ".jpg")
    shutil.copy(train_dogs[0], DATASET_DIR + "train/dogs/")
    shutil.copy(train_cats[0], DATASET_DIR + "train/cats/")
    
for i in range(12000,12500):
    test_dogs = glob.glob(IMGS_ORIGIN + "train/dog." + str(i) + ".jpg")
    test_cats = glob.glob(IMGS_ORIGIN + "train/dog." + str(i) + ".jpg")
    shutil.copy(test_dogs[0], DATASET_DIR + "test/dogs/")
    shutil.copy(test_cats[0], DATASET_DIR + "test/cats/")

In [21]:
for i in range(12000,12500):
    test_dogs = glob.glob(IMGS_ORIGIN + "train/dog." + str(i) + ".jpg")
    test_cats = glob.glob(IMGS_ORIGIN + "train/dog." + str(i) + ".jpg")
    shutil.copy(test_dogs[0], DATASET_DIR + "validation/dogs/")
    shutil.copy(test_cats[0], DATASET_DIR + "validation/cats/")

In [23]:
# Verify Dataset

train_dogs_imgs = glob.glob(TRAIN_DOGS_PATH + "*.jpg")
train_cats_imgs = glob.glob(TRAIN_CATS_PATH + "*.jpg")
validation_dogs_imgs = glob.glob(VALIDATION_DOGS_PATH + "*.jpg")
validation_cats_imgs = glob.glob(VALIDATION_CATS_PATH + "*.jpg")
test_dogs_imgs = glob.glob(TEST_DOGS_PATH + "*.jpg")
test_cats_imgs = glob.glob(TEST_CATS_PATH + "*.jpg")
print(len(train_dogs_imgs), len(train_cats_imgs), len(validation_dogs_imgs), len(validation_cats_imgs), len(test_dogs_imgs), len(test_cats_imgs),)

12000 12000 500 500 500 500


In [34]:
# Learn

def construct_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64,(3,3),activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128,(3,3),activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128,(3,3),activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512,activation="relu"))
    model.add(layers.Dense(1,activation="sigmoid"))

    model.summary()

    model.compile(loss="binary_crossentropy",
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=["acc"])
    preprocess_imgs(model)


def preprocess_imgs(model):
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(150,150),
        batch_size=20,
        class_mode="binary"
    )
    validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(150,150),
        batch_size=20,
        class_mode="binary"
    )
    print(train_generator.class_indices, validation_generator.class_indices)
    learn(model, train_generator, validation_generator)


def learn(model, train_generator, validation_generator):
    history = model.fit_generator(train_generator,
                                  steps_per_epoch=600,
                                  epochs=10,
                                  validation_data=validation_generator,
                                  validation_steps=500)
    model.save('./dogorcat.h5')
    show_result(history)


def show_result(history):
    acc = history.history["acc"]
    val_acc = history.history["val_acc"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs = range(1,len(acc) + 1)

    plt.plot(epochs, acc,"bo",label="Training Acc")
    plt.plot(epochs, val_acc,"b",label="Validation Acc")
    plt.legend()

    plt.figure()

    plt.plot(epochs,loss,"bo",label="Training Loss")
    plt.plot(epochs,val_loss,"b",label="Validation Loss")
    plt.legend()

    plt.show()


if __name__ == "__main__":
    construct_model()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 15, 15, 128)      

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
 
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
 
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

ModuleNotFoundError: No module named 'tensorflow'